## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [1]:
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()

mysql_password = os.getenv("MYSQL_PASSWORD")

cnx  = mysql.connector.connect(
    host = "localhost",
    user = "julianasvieira",
    password = mysql_password
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Esse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python.

In [2]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [4]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [5]:
import pandas as pd

df_livros = pd.read_csv("/home/juliana/Documentos/pipeline-python-mongo-mysql/data/tabela_livros.csv")
df_livros.head()

_id                      Produto Categoria do Produto  \
0  67377dd17c7feba07c385d71          Modelagem preditiva               livros   
1  67377dd17c7feba07c385d72     Iniciando em programação               livros   
2  67377dd17c7feba07c385d84     Iniciando em programação               livros   
3  67377dd17c7feba07c385d92  Ciência de dados com python               livros   
4  67377dd17c7feba07c385d94  Ciência de dados com python               livros   

   Preço     Frete Data da Compra          Vendedor Local da compra  \
0  92.45  5.609697     2020-01-01      Thiago Silva              BA   
1  43.84  0.000000     2020-01-01  Mariana Ferreira              SP   
2  63.25  3.894137     2022-01-01     Juliana Costa              RJ   
3  86.13  5.273176     2021-02-01    Camila Ribeiro              RJ   
4  72.75  1.458158     2021-02-01    Beatriz Moraes              PR   

   Avaliação da compra Tipo de pagamento  Quantidade de parcelas  Latitude  \
0                    1    cartao_credito                       3    -13.29   
1                    5    cartao_credito                       1    -22.19   
2                    5    cartao_credito                       4    -22.25   
3                    4    cartao_credito                       3    -22.25   
4                    4    cartao_credito                       2    -24.89   

   Longitude  
0     -41.71  
1     -48.79  
2     -42.66  
3     -42.66  
4     -51.55

In [6]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [7]:
df_livros.shape

(742, 13)

In [8]:
cursor.execute("""            
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
                id VARCHAR(100),
                Produto VARCHAR(100),
                Categoria_Produto VARCHAR(100),
                Preco FLOAT(10,2),
                Frete FLOAT(10,2),
                Data_Compra DATE,
                Vendedor VARCHAR(100),
                Local_Compra VARCHAR(100),
                Avaliacao_Compra INT,
                Tipo_Pagamento VARCHAR(100),
                Qntd_Parcelas INT,
                Latitude FLOAT(10,2),
                Longitude FLOAT(10,2),
               
                PRIMARY KEY (id)
               
            );
               """)

### Selecionando a base de dados para verificar a tabela criada

In [9]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)

('tb_livros',)
('tb_produtos_2021',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do Dataframe para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

### Dataframe de livros:

In [10]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('67377dd17c7feba07c385d71', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('67377dd17c7feba07c385d72', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('67377dd17c7feba07c385d84', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('67377dd17c7feba07c385d92', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('67377dd17c7feba07c385d94', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('67377dd17c7feba07c385d97', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('67377dd17c7feba07c385d

In [11]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('67377dd17c7feba07c385d71',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('67377dd17c7feba07c385d72',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('67377dd17c7feba07c385d84',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('67377dd17c7feba07c385d92',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('67377dd17c7feba07c385d94',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('67377dd17c7feba07c385d97',
  'Iniciando em progr

In [12]:
# sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

# cursor.executemany(sql, lista_dados)
# cnx.commit()

In [13]:
print(cursor.rowcount, "dados inseridos")

2 dados inseridos


### Dataframe de vendas a partir de 2021:

In [ ]:
df_produtos = pd.read_csv("/home/juliana/Documentos/pipeline-python-mongo-mysql/data/tabela_2021_em_diante.csv")
df_produtos.head()

_id            Produto Categoria do Produto    Preço  \
0  67377dd17c7feba07c385d7a  Xadrez de madeira           brinquedos    25.23   
1  67377dd17c7feba07c385d7b         Impressora          eletronicos   322.04   
2  67377dd17c7feba07c385d7c     Mesa de centro               moveis   282.22   
3  67377dd17c7feba07c385d7d        Tablet ABXY          eletronicos  1100.42   
4  67377dd17c7feba07c385d7e              Fogão     eletrodomesticos   791.81   

       Frete Data da Compra        Vendedor Local da compra  \
0   0.000000     2021-01-01    Thiago Silva              BA   
1  14.732100     2021-01-01      João Souza              SP   
2  12.611805     2021-01-01      João Souza              SP   
3  68.451348     2021-01-01  Beatriz Moraes              SP   
4  42.444626     2021-01-01   Juliana Costa              SP   

   Avaliação da compra Tipo de pagamento  Quantidade de parcelas  Latitude  \
0                    5    cartao_credito                       2    -13.29   
1                    3    cartao_credito                       1    -22.19   
2                    3            boleto                       1    -22.19   
3                    1             cupom                       1    -22.19   
4                    4            boleto                       1    -22.19   

   Longitude  
0     -41.71  
1     -48.79  
2     -48.79  
3     -48.79  
4     -48.79

In [21]:
df_produtos.shape

(6574, 13)

In [20]:
df_produtos.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [17]:
cursor.execute("""            
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_produtos_2021(
                id VARCHAR(100),
                Produto VARCHAR(100),
                Categoria_Produto VARCHAR(100),
                Preco FLOAT(10,2),
                Frete FLOAT(10,2),
                Data_Compra DATE,
                Vendedor VARCHAR(100),
                Local_Compra VARCHAR(100),
                Avaliacao_Compra INT,
                Tipo_Pagamento VARCHAR(100),
                Qntd_Parcelas INT,
                Latitude FLOAT(10,2),
                Longitude FLOAT(10,2),
               
                PRIMARY KEY (id)
               
            );
               """)

In [18]:
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)

('tb_livros',)
('tb_produtos_2021',)


In [22]:
lista_dados = [tuple(row) for i, row in df_produtos.iterrows()]
lista_dados

[('67377dd17c7feba07c385d7a',
  'Xadrez de madeira',
  'brinquedos',
  25.23,
  0.0,
  '2021-01-01',
  'Thiago Silva',
  'BA',
  5,
  'cartao_credito',
  2,
  -13.29,
  -41.71),
 ('67377dd17c7feba07c385d7b',
  'Impressora',
  'eletronicos',
  322.04,
  14.7321002401,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('67377dd17c7feba07c385d7c',
  'Mesa de centro',
  'moveis',
  282.22,
  12.6118050347,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('67377dd17c7feba07c385d7d',
  'Tablet ABXY',
  'eletronicos',
  1100.42,
  68.4513476183,
  '2021-01-01',
  'Beatriz Moraes',
  'SP',
  1,
  'cupom',
  1,
  -22.19,
  -48.79),
 ('67377dd17c7feba07c385d7e',
  'Fogão',
  'eletrodomesticos',
  791.81,
  42.4446263246,
  '2021-01-01',
  'Juliana Costa',
  'SP',
  4,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('67377dd17c7feba07c385d7f',
  'Violão',
  'instrumentos musicais',
  316.84,
  17.1595536296,
  '2022-01-01',
  '

In [ ]:
# sql = "INSERT INTO dbprodutos.tb_produtos_2021 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

# cursor.executemany(sql, lista_dados)
# cnx.commit()

In [24]:
print(cursor.rowcount, "dados inseridos")

6574 dados inseridos


## Visualizando os dados inseridos

In [25]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros;")

for row in cursor:
    print(row)

('67377dd17c7feba07c385d71', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('67377dd17c7feba07c385d72', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('67377dd17c7feba07c385d84', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('67377dd17c7feba07c385d92', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('67377dd17c7feba07c385d94', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('67377dd17c7feba07c385d97', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [26]:
cursor.close()

True

In [27]:
cnx.close()